In [13]:
# Importing Libraries
import os
import openai
from typing import List
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from fastapi import FastAPI, UploadFile, File
from llama_index.core import  VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_cpp import Llama
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


In [18]:
from langchain_community.embeddings import LlamaCppEmbeddings



In [19]:
llama = LlamaCppEmbeddings(model_path=r"D:\DigiMark\Chat-with-Doc-LLAMA-Langcahin\models\llama-2-7b-chat.gguf.q4_0.bin")



llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from D:\DigiMark\Chat-with-Doc-LLAMA-Langcahin\models\llama-2-7b-chat.gguf.q4_0.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = llama-2-7b-chat.ggmlv3.q8_0.bin
llama_model_loader: - kv   2:                        general.description str              = converted from legacy GGJTv3 MOSTLY_Q...
llama_model_loader: - kv   3:                          general.file_type u32              = 7
llama_model_loader: - kv   4:                       llama.context_length u32              = 2048
llama_model_loader: - kv   5:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   6:                          llama.block_coun

llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  17:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  18:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - type q8_0:  226 tensors
llm_load_vocab: special tokens definition check successful ( 259/32000 ).
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = SPM
llm_load_print_meta: n_vocab          = 32000
llm_load_print_meta: n_merges         = 0
llm_load_print_meta: n_ctx_train      = 2048
llm_lo

In [21]:
text = "This is a test document."



In [22]:
query_result = llama.embed_query(text)




llama_print_timings:        load time =   40813.42 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =   40791.43 ms /     7 tokens ( 5827.35 ms per token,     0.17 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   40814.77 ms /     8 tokens


4096


In [9]:
import llama_index.embeddings

In [3]:
llm_embed = Llama(model_path = "models/llama-2-7b-chat.gguf.q4_0.bin",verbose=False,embedding=True)

In [11]:
from llama_index.llms.llama_cpp import LlamaCPP

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"

model_url = "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_0.gguf"

# llm = LlamaCPP(
#     # model_url=model_url,
#     model_path='models/llama-2-7b-chat.gguf.q4_0.bin',
#     temperature=0.1,
#     max_new_tokens=256,
#     context_window=3900,
#     generate_kwargs={},
#     model_kwargs={"n_gpu_layers": -1},  # if compiled to use GPU
#     messages_to_prompt=messages_to_prompt,
#     completion_to_prompt=completion_to_prompt,
#     verbose=False,
# )

llm = Llama(
    # model_url=model_url,
    model_path='models/llama-2-7b-chat.gguf.q4_0.bin',
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},  # if compiled to use GPU
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
    embedding=True
)


response = llm.complete("Hello, how are you?")
print(str(response))

AttributeError: 'Llama' object has no attribute 'complete'

In [6]:
help(llm.chat)

Help on method chat in module llama_index.llms.llama_cpp.base:

chat(messages: Sequence[llama_index.core.base.llms.types.ChatMessage], **kwargs: Any) -> llama_index.core.base.llms.types.ChatResponse method of llama_index.llms.llama_cpp.base.LlamaCPP instance
    Chat endpoint for LLM.
    
    Args:
        messages (Sequence[ChatMessage]):
            Sequence of chat messages.
        kwargs (Any):
            Additional keyword arguments to pass to the LLM.
    
    Returns:
        ChatResponse: Chat response from the LLM.
    
    Examples:
        ```python
        from llama_index.core.llms import ChatMessage
    
        response = llm.chat([ChatMessage(role="user", content="Hello")])
        print(response.content)
        ```



In [43]:
x = llm.embed('apple')

In [32]:
from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str=..., query_str=...)

# or easily convert to message prompts (for chat API)
messages = qa_template.format_messages(context_str='apple', query_str='what is apple')

In [29]:
dir(llama_index.llms)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'ollama']

In [2]:
load_dotenv() # Loading Enviroment Variables
pc = Pinecone(api_key=os.environ.get('PINE_CONE_API_KEY')) # Setting Pine Cone API Key



In [ ]:
vector_store = PineconeVectorStore(pinecone_index=index, api_key=os.environ.get('PINE_CONE_API_KEY'), namespace='mohsin-cv-namespace')


Help on package llama_index.llms in llama_index:

NAME
    llama_index.llms

PACKAGE CONTENTS
    openai (package)

FILE
    (built-in)


